# Tableau and player form

Here we create a csv file that is used for making further Tableau visualizations. 

In [1]:
# recent gameweek
gameweek = 38

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch player stats data
all_df = pd.read_csv('../data/fpl/data_week1.csv', index_col=0)
all_df['gameweek'] = 1
all_df['xPoints'] = all_df['adjusted points']

for week in range(2,gameweek+1):
    filepath = '../data/fpl/data_week' + str(week) + '.csv'
    df = pd.read_csv(filepath, index_col=0)
    df['gameweek'] = week
    column_name = 'xPoints week ' + str(week)
    df['xPoints'] = df[column_name]
    all_df = all_df.append(df,sort='False')

all_df.shape

(24334, 356)

In [4]:
# calculate player form
for i in [5,10,15,20]:
    gameweeks = np.arange(gameweek-i+1,gameweek+1)
    column_name = 'form ' + str(i)
    all_df.loc[all_df['gameweek']==gameweek, column_name] = (all_df[all_df['gameweek'].isin(gameweeks)].groupby(['id'])['xPoints'].sum() / (all_df.loc[(all_df['gameweek']==gameweek),'games played'].round() - all_df.loc[(all_df['gameweek']==gameweek-i),'games played'].round()) )
    # dirty fix in case there are inf results
    all_df.loc[(all_df['gameweek']==gameweek) & np.isinf(all_df[column_name]), column_name] = np.nan

In [5]:
# top form players
all_df[all_df['gameweek']==gameweek][['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10', 'form 15', 'form 20']].sort_values(by=['form 10'], ascending=False).head(20)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10,form 15,form 20
id,,,,,,,,,,
254,Salah,37.258065,231,6.2,228.457156,6.131751,7.797904,7.388823,6.181187,5.865449
272,De Bruyne,25.178571,141,5.6,160.932219,6.391634,8.843582,7.206951,5.428333,5.267131
259,Alexander-Arnold,36.363636,160,4.4,175.425703,4.824207,6.092858,6.846809,5.820315,5.363331
388,Kane,35.072464,242,6.9,221.074696,6.303369,5.854210,6.385672,6.099397,6.049498
481,Pereira,33.260870,153,4.6,132.277550,3.976972,4.130071,6.381081,5.196919,5.159798
232,Chilwell,27.254902,139,5.1,128.413809,4.711586,5.445178,6.271638,5.560255,4.905618
573,Olsen,6.944444,25,3.6,24.220094,3.487693,NaN,6.001046,4.354025,3.885223
290,Mata,8.888889,32,3.6,29.661021,3.336865,5.875262,5.875262,4.250174,4.250174
251,Mané,35.200000,176,5.0,188.267615,5.348512,6.413850,5.829796,5.000834,5.256099


In [6]:
# save data
filepath = '../data/fpl/all_data.csv'
all_df.to_csv(filepath)

filepath = '../data/fpl/data_week' + str(gameweek) + '.csv'
all_df[all_df['gameweek']==gameweek].to_csv(filepath)